# Bulk loading data
In this example we're going to bulk load data into Neptune from S3. 



### Pre-reqs

As a pre-requiste we need to create a S3 bucket and upload our content. For this demo we're using https://github.com/aws-samples/amazon-neptune-samples/tree/master/gremlin/collaborative-filtering/data

We also need to make sure Neptune has a role that allows it to list and read from the S3 bucket and that there is an S3 endpoint in the VPC to allow private access to the bucket

### Bulk load data

Next we'll use the bulk loader API to tell Neptune to upload content

In [ ]:
import requests
import json
neptune_endpoint = 'https://neptunedbcluster-8booe3qn0pin.cluster-cjsktiz5uanu.eu-west-2.neptune.amazonaws.com:8182/loader'
s3_bucket = 's3://cjl-temp-eu-west-2/'
iam_neptune_s3_role_arn = 'arn:aws:iam::008369042577:role/Neptune-SageMaker-Tutorial-N-NeptuneLoadFromS3Role-3HBYVXB7IMKX'
headers = {'Content-Type': 'application/json'}
json_data = {}
json_data['source'] = s3_bucket
json_data['iamRoleArn'] = iam_neptune_s3_role_arn
json_data['format'] = 'csv'
json_data['region'] = 'eu-west-2'
json_data['failOnError'] = 'FALSE'

r = requests.post(neptune_endpoint, data=json.dumps(json_data), headers=headers)
print(r.status_code)
print(json.loads(r.text)['payload']['loadId'])

### Validate upload

We can see the upload request was successfully accepted as shown by the 200 status code and the load ID. 

We can use the load ID to now check the status of the upload.

In [ ]:
import requests
neptune_endpoint = 'https://neptunedbcluster-8booe3qn0pin.cluster-cjsktiz5uanu.eu-west-2.neptune.amazonaws.com:8182/loader'
load_id = '59f1ae12-3f1f-4ffa-b7ca-659bf6d5d781'

r = requests.get(neptune_endpoint + '?loadId=' + load_id)
print(r.status_code)
print(r.text)

### Validate results

We'll now validate the results by querying some of the data we uploaded.

In [ ]:
%run '../util/neptune.py' 
g = neptune.graphTraversal() 
g.V().hasId('Luke').valueMap(). \
next()

In [ ]:
g.V().hasLabel('game').groupCount().by("Platform"). \
next()

In [ ]:
g.V().hasLabel('game').groupCount().by("GameGenre"). \
next()